In [11]:
import os

In [12]:
%pwd

'/Users/rahul/Documents/TextSum'

In [23]:
os.chdir("TextSum")

In [24]:
%pwd

'/Users/rahul/Documents/TextSum'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [16]:
from src.TextSum.constants import *
from src.TextSum.utils.common import read_yaml,create_directories

In [17]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [18]:
import os
from src.TextSum.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [19]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['document'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_edin= load_from_disk(self.config.data_path)
        ds_pt = {
            split: dataset_edin[split].map(convert_examples_to_features, batched=True)
            for split in dataset_edin
        }
        ds_pt.save_to_disk(os.path.join(self.config.root_dir,"edin_data"))


In [26]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-08-18 13:17:08,193:INFO:common:yaml file: config/config.yaml loaded succesfully]
[2024-08-18 13:17:08,195:INFO:common:yaml file: params.yaml loaded succesfully]
[2024-08-18 13:17:08,195:INFO:common:Created directory: artifacts]
[2024-08-18 13:17:08,196:INFO:common:Created directory: artifacts/data_transformation]


FileNotFoundError: Directory artifacts/data_ingestion/edin_data/edin_data is neither a `Dataset` directory nor a `DatasetDict` directory.